In [1]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage.io import imread
#from sklearn.externals
import joblib
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import cv2
import os
import glob
from PIL import Image # This will be used to read/modify images (can be done via OpenCV too)
from numpy import *
import sklearn
import skimage

In [2]:
!pip install "opencv-contrib-python>=4.5.1"
print(sklearn.__version__, 'sklean')
print(skimage.__version__)
print(cv2.__version__)

Defaulting to user installation because normal site-packages is not writeable
0.23.2 sklean
0.17.2
4.5.4


In [3]:
# define parameters of HOG feature extraction
orientations = 9
pixels_per_cell = (16, 16)
cells_per_block = (2, 2)

In [4]:
# define path to images:

pos_im_path = r"/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/dataset/bigger_dataset_128_64/pos"  
neg_im_path = r"/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/dataset/bigger_dataset_128_64/neg"

#pos_im_path = '/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/bigger_dataset_90_64/pos_cropped'
#neg_im_path = '/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/bigger_dataset_90_64/neg_cropped'

#pos_im_path = r"/media/dtu-project2/2GB_HDD/classifymito/mitosis_detection/MandNM/mitosis"
#neg_im_path = r"/media/dtu-project2/2GB_HDD/classifymito/mitosis_detection/MandNM/nonmitosis"

#pos_im_path = r"/media/dtu-project2/2GB_HDD/classifymito/mitosis_detection/with_nucleus"
#neg_im_path = r"/media/dtu-project2/2GB_HDD/classifymito/mitosis_detection/no_nucleus"

In [5]:
# read the image files:
pos_im_listing = os.listdir(pos_im_path) # it will read all the files in the positive image path (so all the required images)
neg_im_listing = os.listdir(neg_im_path)
num_pos_samples = size(pos_im_listing) # simply states the total no. of images
num_neg_samples = size(neg_im_listing)
print(num_pos_samples) # prints the number value of the no.of samples in positive dataset
print(num_neg_samples)


2699
2545


Hog feature calcuation

In [6]:
# compute HOG features and label them:
data= []
labels = []
kernel = np.ones((10,10), np.float32)/100

for file in pos_im_listing: #this loop enables reading the files in the pos_im_listing variable one by one
    filename = os.path.join(pos_im_path, file)
    #print(file)
    img = cv2.imread(filename)
    img = cv2.filter2D(img, -1, kernel)
    #img = img.resize((240,240))
    #img = img.resize((64,128))
    #gray = img.convert('L') # convert the image into single channel i.e. RGB to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #print((np.ndarray(gray)).shape)
    # calculate HOG for positive features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    #print(fd.shape)
    data.append(fd)
    #print(size(data))
    labels.append(1)
    #print(labels)

## for negative images
for file in neg_im_listing: #this loop enables reading the files in the pos_im_listing variable one 
    #print(file)
    filename = os.path.join(neg_im_path, file)
    #print(file)
    img = cv2.imread(filename)
    img = cv2.filter2D(img, -1, kernel)
    #img=Image.open(neg_im_path + '\\' + file)
    #img = img.resize((240,240))
    #img = img.resize((64,128))
    #gray= img.convert('L')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Now we calculate the HOG for negative features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True, transform_sqrt = True) 
    #print(fd.shape)
    data.append(fd)
    labels.append(0)

shape(data)


(5244, 756)

Beblid descriptor calculation

In [158]:
# beblid feature using ORB detector
detector = cv2.ORB_create(1000)

descriptor = cv2.ORB_create()
#descriptor = cv2.xfeatures2d.BEBLID_create(0.75, None)

mitosis_desc = []
nonmitosis_dec = []
max_des_row = 0
temp_list = []
labels_beblid = []

fd_beblid = []
count = 0

#######
for file in pos_im_listing: #this loop enables reading the files in the pos_im_listing variable
    filename = os.path.join(pos_im_path, file)
    img = cv2.imread(filename)
    
    #print('ok')
    #print(img.shape)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #print(gray.shape)
    #print(filename)
    kpts = detector.detect(gray, None)
    kpts, desc = descriptor.compute(gray, kpts)

    desc = np.reshape(desc, (desc.shape[0]*desc.shape[1], 1), order='C')
    mitosis_desc.append(desc)
    labels_beblid.append(1)
    temp = desc.shape[0]
    #print(temp)
    if temp > max_des_row:
        max_des_row = temp

for file in neg_im_listing: #this loop enables reading the files in the pos_im_listing variable
    filename = os.path.join(neg_im_path, file)
    img = cv2.imread(filename)
    
    #print('ok')
    #print(img.shape)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #print(gray.shape)
    #print(filename)
    kpts = detector.detect(gray, None)
    kpts, desc = descriptor.compute(gray, kpts)

    desc = np.reshape(desc, (desc.shape[0]*desc.shape[1], 1), order='C')
    mitosis_desc.append(desc)
    labels_beblid.append(0)
    temp = desc.shape[0]
    #print(temp)
    if temp > max_des_row:
        max_des_row = temp
    
    #print(temp)
    #print(desc.shape)
    #print(shape(kpts))
print(max_des_row)
##print(mitosis_desc[0])

for row in mitosis_desc:
    row = np.reshape(row, (1, row.shape[0]*row.shape[1]))
    #row = np.transpose(row)
    
    row = list(row[0])
    row_len = len(row)
    #print(row_len, 'before')
    
    pad_left  = (max_des_row-row_len)//2
    pad_right = max_des_row-row_len - pad_left

    row = np.pad(row, pad_width=(pad_left, pad_right), mode='constant')
    fd_beblid.append(row)
    #print('feature number: ', count)
    count += 1


print(shape(fd_beblid))
#print(len(fd_beblid))
#print(len(fd_beblid[1]))
#print(fd_beblid[1])




AttributeError: 'NoneType' object has no attribute 'shape'

In [7]:
np_data = np.array(data)
print(np_data.shape)

np_data_mito = np.array(fd_beblid)
print(np_data_mito.shape)

(5244, 756)


NameError: name 'fd_beblid' is not defined

In [8]:
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels) 
type(labels)
print(labels.shape)

labels_beblid = le.fit_transform(labels_beblid) 
type(labels_beblid)
print(labels_beblid.shape)

(5244,)


NameError: name 'labels_beblid' is not defined

In [9]:
## partition data into train and test data

data_to_use = np_data
#data_to_use = np_data_mito

label_to_use = labels
#label_to_use = labels_beblid

(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(data_to_use), label_to_use, test_size=0.2, random_state=10, stratify=label_to_use)

print('train data: \t', trainData.shape)
print('test data: \t', testData.shape)


train data: 	 (4068, 756)
test data: 	 (1018, 756)


In [169]:
#%%
# Partitioning the data into training and testing splits, using 80%
# of the data for training and the remaining 20% for '''s

'''
print(" Constructing training/testing split...")
#(trainData, testData, trainLabels, testLabels) = train_test_split(
#	np.array(data_to_use), label_to_use, test_size=0.20, random_state=10, stratify=label_to_use)

(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(data_to_use), label_to_use, test_size=0.2, random_state=10, stratify=label_to_use)


print('train data: \t', trainData.shape)
print('test data \t', testData.shape)

s=  np.sum(testLabels)
test_label_per = s*100/len(testLabels)
print(test_label_per)

s=  np.sum(trainLabels)
train_label_per = s*100/len(trainLabels)
print(train_label_p
''''per)

SyntaxError: EOL while scanning string literal (<ipython-input-169-d6c65bd6f1f2>, line 22)

In [10]:
#%% Train the linear SVM
from sklearn.svm import LinearSVC, SVC
print(" Training Linear SVM classifier...")
model  = LinearSVC(max_iter=5000)
model.fit(trainData, trainLabels)
print('training finished')


Training Linear SVM classifier...
training finished


In [11]:
#%% Evaluate the classifier######################################################################
print(" Evaluating classifier on test data ...")
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))

Evaluating classifier on test data ...
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       490
           1       0.96      0.97      0.96       528

    accuracy                           0.96      1018
   macro avg       0.96      0.96      0.96      1018
weighted avg       0.96      0.96      0.96      1018



In [12]:
# Save the model:
#%% Save the Model

%cd /media/dtu-project2/2GB_HDD/Detection_HOG_SVM/trained_models/hog_models_128_64 
joblib.dump(model, 'hogsvm_pedes_'+str(orientations)+'bins_756_feat.npy')
#joblib.dump(model, 'hogsvm_mito_vs_non_mito_'+str(orientations)+'bins.npy', protocol=2)

/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/trained_models/hog_models_128_64


['hogsvm_pedes_9bins_756_feat.npy']

In [21]:
### visualizing model output for some of the images form test data
import os
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.metrics import precision_score, recall_score, f1_score

extra__pos_data_dir = '/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/neg_pos_imgs_for_hog_svm_unsen/pos'
extra__neg_data_dir = '/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/neg_pos_imgs_for_hog_svm_unsen/neg'
pos_pred = []
neg_pred = []
test_data = []
positive_decision_boundary = 0
negative_decision_boundary = 0

for img in os.listdir(extra__pos_data_dir):
    file_path = os.path.join(extra__pos_data_dir, img)
    temp_img = Image.open(file_path)
    gray_img = temp_img.convert('L')
    #plt.figure()
    #plt.imshow(gray_img)
    fd,hog_feat = hog(gray_img, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True,visualize=True)
    test_data.append(fd)
    #plt.figure()
    #plt.imshow(hog_feat)
    #plt.show()
    fd = np.reshape(fd,(1,fd.shape[0]))
    #model_out = model.predict(fd)
    model_dec_function = model.decision_function(fd)
    ### decision boundary
    if model_dec_function >= positive_decision_boundary:
        model_out = 1
    else:
        model_out = 0

    pos_pred.append(model_out)
    print(model_out, model_dec_function)

for img in os.listdir(extra__neg_data_dir):
    file_path = os.path.join(extra__neg_data_dir, img)
    temp_img = Image.open(file_path)
    gray_img = temp_img.convert('L')
    #plt.figure()
    #plt.imshow(gray_img)
    fd,hog_feat = hog(gray_img, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True,visualize=True)
    test_data.append(fd)
    #plt.figure()
    #plt.imshow(hog_feat)
    #plt.show()
    fd = np.reshape(fd,(1,fd.shape[0]))
    #model_out = model.predict(fd)
    model_dec_function = model.decision_function(fd)
    ##### 
    if model_dec_function <= negative_decision_boundary:
        model_out = 0
    else:
        model_out = 1
    neg_pred.append(model_out)
    print(model_out, model_dec_function)

print(neg_pred)



FileNotFoundError: [Errno 2] No such file or directory: '/media/dtu-project2/2GB_HDD/Detection_HOG_SVM/neg_pos_imgs_for_hog_svm_unsen/pos'

In [38]:
test_data = np.array(test_data)
test_data.shape
pred = pos_pred+neg_pred
pred = np.array(pred)
print(pred)

pos_label = [1]*len(pos_pred)
neg_label = [0]*len(neg_pred)
test_labels = np.array(pos_label+neg_label)
print(test_labels)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


In [12]:
print(" Evaluating classifier on test data ...")
predictions = model.predict(test_data)
print(classification_report(test_labels, pred))

Evaluating classifier on test data ...


NameError: name 'test_data' is not defined